### Becoming a backprop ninja! 

In [42]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [43]:
words = open('../lecture 2/names.txt').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [44]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [45]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [46]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [47]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [48]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [49]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3528, grad_fn=<NegBackward0>)

In [ ]:
#Time for manual backprop! 

In [50]:
#Step 1: how does loss get affected by logprobs? 
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n #we've derived that the gradient will be -1/n for the examples in logprobs that are actually chosen. naturally the ones that are not chosen
                                #have a gradient of 0, so we init a 0 tensor and just put in -1/n with our smart indexing. 
#let's see if this is right:
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


In [ ]:
#Step 2: how does loss get affected by probs?
# lobprobs = ln(probs), elementwise we have that our derivatives just follow the rule for ln(x) so we just have 1/x.
dprobs = probs**-1 * dlogprobs #multiply by global derivative dlogprobs (so far) to follow chain rule, dummy
cmp('probs', dprobs, probs) #yeehaw

probs           | exact: True  | approximate: True  | maxdiff: 0.0


In [ ]:
#Step 3: how does loss get affected by counts and couns_sum_inv? 
#c = a * b
# but also a broadcasting thing in our case. we have 2 operations happening under the hood when we do probs = counts * counts_sum_inv. we have the multiplication and a broadcasting
dcounts_sum_inv = (counts * dprobs).sum(1,keepdim=True) #we're summing across the rows because of the multivariate chain rule here. we're broadcasting a 32x1 over a 32x27
                                                        #meaning we're replicating the 32x1 column vector over all 27 rows of the counts matrix. essentially each element of the thing
                                                        #is being used 27 times, and these all have gradients that we're going to sum up. keeping dims so shit works. idk.
dcounts = (counts_sum_inv * dprobs) #no need for the sum as this broadcasts just fine on it's own
cmp("dcounts_sum_inv", dcounts_sum_inv, counts_sum_inv) #Works just fine baby-boo
cmp("counts", dcounts, counts_sum_inv) #This shit don't work. counts here is being used for multiple nodes, counts calculates counts_sum which is then used to calculate counts_sum_inv, and probs
                                       #also uses counts in it's calculation. we need to finish working backwards and propagate through counts_sum first.

dcounts_sum_inv | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: False | approximate: False | maxdiff: 0.5347182750701904


In [68]:
#Step 4: how does loss get affected by counts_sum? 
#counts_sum_inv = 1/counts_sum. derivative of 1/x is -1/x^2. so we just do that 
dcounts_sum = (-1/(counts_sum**2) ) * dcounts_sum_inv #Remember the chain rule dumbass
cmp("dcounts_sum", dcounts_sum, counts_sum)

dcounts_sum     | exact: True  | approximate: True  | maxdiff: 0.0


In [70]:
counts.shape , counts_sum.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
#Step 5: how does loss get affected by counts? 
dcounts += torch.ones_like(counts) * dcounts_sum #Dawg i don't even know what we're doing here. revisit this! but we're adding the remaining partial chainrule gradient for the 
                                                #rest of dcounts so it should be working fine now. 
cmp("dcounts", dcounts, counts)

dcounts         | exact: True  | approximate: True  | maxdiff: 0.0
